# Question 1

## How do levels of particulate matter (PM2.5) affect asthma?

### pm2.5 and Asthma datasets cleanup

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
#load datasets 
asthma_data = pd.read_csv('U.S._Chronic_Disease_Indicators__Asthma.csv')
pm25_data = pd.read_csv('Daily_Census_Tract-Level_PM2.5_Concentrations__2011-2014.csv')

In [3]:
#clean the asthma datasetn
#the dataset to only include YearStart, Location Abbr, StratificationCategoryID1, StratificationID1, DataValueUnit, DataValueType, and DataValue 
asthma_data = asthma_data[['YearStart', 'LocationDesc', 'LocationAbbr', 'DataValueUnit', 
                   'DataValueType', 'DataValue', 'StratificationCategoryID1', 'StratificationID1']]
#asthma_data.head()

In [4]:
#clean the asthma dataset so that..
##it only includes the rows that show cases per 10,000
##stratification ID1 = Gender
##only shows age adjusted rates 
##data value must be a number

asthma_data = asthma_data[asthma_data['DataValueUnit'] == 'cases per 10,000']
asthma_data = asthma_data[asthma_data['DataValueType'] == 'Age-adjusted Rate']
asthma_data = asthma_data[asthma_data['StratificationCategoryID1'] == 'GENDER']
asthma_data = asthma_data[pd.to_numeric(asthma_data['DataValue'], errors='coerce').notnull()]


In [5]:
#include state fips code in asthma dataset

##create a FIPS dict
fips_dict = {
    'Alabama': '01',
    'Alaska': '02',
    'Arizona': '04',
    'Arkansas': '05',
    'California': '06',
    'Colorado': '08',
    'Connecticut': '09',
    'Delaware': '10',
    'District of Columbia': '11',
    'Florida': '12',
    'Georgia': '13',
    'Hawaii': '15',
    'Idaho': '16',
    'Illinois': '17',
    'Indiana': '18',
    'Iowa': '19',
    'Kansas': '20',
    'Kentucky': '21',
    'Louisiana': '22',
    'Maine': '23',
    'Maryland': '24',
    'Massachusetts': '25',
    'Michigan': '26',
    'Minnesota': '27',
    'Mississippi': '28',
    'Missouri': '29',
    'Montana': '30',
    'Nebraska': '31',
    'Nevada': '32',
    'New Hampshire': '33',
    'New Jersey': '34',
    'New Mexico': '35',
    'New York': '36',
    'North Carolina': '37',
    'North Dakota': '38',
    'Ohio': '39',
    'Oklahoma': '40',
    'Oregon': '41',
    'Pennsylvania': '42',
    'Rhode Island': '44',
    'South Carolina': '45',
    'South Dakota': '46',
    'Tennessee': '47',
    'Texas': '48',
    'Utah': '49',
    'Vermont': '50',
    'Virginia': '51',
    'Washington': '53',
    'West Virginia': '54',
    'Wisconsin': '55',
    'Wyoming': '56'
}

##map asthma dataset to fips dict
asthma_data['FIPS_Code'] = asthma_data['LocationDesc'].map(fips_dict)
#asthma_data.head()

In [6]:
#only include years 2011 to 2014 in Asthma dataset because that is the only data provided in the pm2.5 dataset
asthma_data = asthma_data[(asthma_data['YearStart'] >= 2011) & (asthma_data['YearStart'] <= 2014)]
#asthma_data.head()

In [7]:
#clean asthma dataset to only include what is needed
asthma_data = asthma_data.drop(['LocationDesc', 'LocationAbbr', 'DataValueUnit', 
                                'DataValueType', 'StratificationCategoryID1'], axis = 1)


In [8]:
#clean pm2.5 dataset to only include whats needed
pm25_data = pm25_data.drop(['date', 'countyfips', 'ctfips', 'latitude', 'longitude', 'year'], axis = 1)

In [9]:
#prepare df for merging
asthma_data['FIPS_Code'] = asthma_data['FIPS_Code'].astype(int)
pm25_data['statefips'] = pm25_data['statefips'].astype(int)
asthma_data = asthma_data.rename(columns = {'FIPS_Code': 'statefips'})
pm25_data.head()

,statefips,ds_pm_pred,ds_pm_stdd
0,37,13.0652,4.2744
1,37,12.9526,4.2139
2,29,13.8699,5.0710
3,37,12.8842,4.2481
4,32,8.1347,3.3742


In [ ]:
#Merge the data
merged_df = asthma_data.merge(pm25_data, on = ("statefips"), how = "left")
merged_df = merged_df[pd.to_numeric(merged_df['ds_pm_pred'], errors='coerce').notnull()]


In [11]:
#convert datasets to csv due to lack of memory 
merged_df.to_csv('Asthma and PM2.5.csv', index=False)
asthma_data.to_csv("Asthma Cleaned Data.csv", index = False)